In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import string 
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_curve, auc, roc_auc_score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Megha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Megha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Megha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
#Load dataset
import pandas as pd
df1 = pd.read_csv("cleanprojectdataset.csv")
df1['Text Label'] = np.where(df1['Text Label']=='Bullying',1,0)

In [4]:
print(df1)

                                                  Tweet  Text Label
0     .omg why are poc wearing fugly blue contacts s...           0
1     .Sorry but most of the runners popular right n...           0
2     .those jeans are hideous, and I?m afraid he?s ...           0
3     .I had to dress up for a presentation in class...           0
4     .Am I the only one who thinks justin bieber is...           0
...                                                 ...         ...
1060  No we are not, But you are a race baiting libt...           1
1061  you wont get anyone for this challenge., after...           1
1062  I will follow you if you are not a libtard,Mus...           1
1063  michaelianblack Ur a child, an ostrich w/ your...           1
1064  FoxNews. not to all the ppl I know that live t...           1

[1065 rows x 2 columns]


In [4]:
#Tokenize words and labels into lists

Tweet = []
Labels = []

for row in df1['Tweet']:
    #tokenize words
    words = word_tokenize(row)
    #remove punctuations
    clean_words = [word.lower() for word in words if word not in set(string.punctuation)]
    #remove stop words
    english_stops = set(stopwords.words('english'))
    characters_to_remove = ["''",'``',"rt","https","’","“","”","\u200b","--","n't","'s","...","//t.c" ]
    clean_words = [word for word in clean_words if word not in english_stops]
    clean_words = [word for word in clean_words if word not in set(characters_to_remove)]
    #Lematise words
    wordnet_lemmatizer = WordNetLemmatizer()
    lemma_list = [wordnet_lemmatizer.lemmatize(word) for word in clean_words]
    Tweet.append(lemma_list)

    for row in df1["Text Label"]:
        Labels.append(row)

In [5]:
#combine them to create bag of words
combined = zip(Tweet, Labels)

In [6]:
#Create bag of words and dictionary object
def bag_of_words(words):
    return dict([(word, True) for word in words])

In [7]:
#Key, Value Pair into new list for modeling
Final_Data = []
for r, v in combined:
    bag_of_words(r)
    Final_Data.append((bag_of_words(r),v))

In [8]:
#random shuffle
import random
random.shuffle(Final_Data)
print(len(Final_Data))
#print(Final_Data)

1065


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df1["Tweet"], 
                                                    df1["Text Label"], 
                                                    random_state=0)

In [10]:
def func_one():
    
    return len(df1[df1['Text Label']==1])/len(df1['Text Label'])*100

func_one()

40.093896713615024

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

def func_two():
    
    # List all tokens and their counts as a dictionary:
    vocabulary = CountVectorizer().fit(X_train).vocabulary_
    
    # You want only the keys, i.e, the words:
    vocabulary = [x for x in vocabulary.keys()]
    
    # Store the lengths in a separate list:
    len_vocabulary = [len(x) for x in vocabulary]
    
    # Use the index of the longest token:
    return vocabulary[np.argmax(len_vocabulary)]

func_two()

'healtheworldin5words'

In [12]:
def func_three():
    
    cv = CountVectorizer().fit(X_train)
    
    # Transform both X_train and X_test with the same CV object:
    X_train_cv = cv.transform(X_train)
    X_test_cv = cv.transform(X_test)
    
    # Classifier for prediction:
    clf = MultinomialNB(alpha=0.1)
    clf.fit(X_train_cv, y_train)
    preds_test = clf.predict(X_test_cv)
    
    return roc_auc_score(y_test, preds_test)

func_three()

0.6522959183673469